셀 1 — 준비

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from arc import Caesium, Rubidium

# ==== 사용자 설정 ====
ATOM = "Cs"                # "Cs" 또는 "Rb"
START_STATE = (14, 1, 1.5) # (n, l, j): 14P3/2 예시
N_FLOOR = 5                # 하향 탐색 시 최소 n' (너무 낮은 레벨만 보면 느려짐 방지)
MAX_STEPS = 12             # 캐스케이드 최대 단계(안정성/속도용)
MIN_WEIGHT = 1e-6          # 너무 작은 가지는 가지치기(속도용)
VISIBLE_RANGE = (350.0, 750.0)  # nm, 보기용
COUNT_WINDOW = (500.0, 600.0)   # nm, 카메라 민감대역 집계용
BIN_WIDTH_NM = 1.0         # 히스토그램 bin 폭 (1 nm 권장)

def make_atom(name="Cs"):
    return Caesium() if name.lower() in ("cs","caesium","cesium") else Rubidium()

def l_to_letter(l):
    return "SPDFGH"[l] if l < 6 else f"L{l}"

def state_str(s):
    n,l,j = s
    return f"{n}{l_to_letter(l)}_{int(2*j)}/2"


셀 2 — E1 허용 하향 전이와 A계수 계산

In [ ]:
def allowed_transitions_with_A(atom, s, n_floor=5):
    """
    상태 s=(n,l,j)에서 가능한 E1 하향 전이 후보와 Einstein A를 나열.
    ARC의 getTransitionRate가 selection rule을 반영해 금지 전이에는 0을 돌려줌.
    반환: 리스트 [(next_state, A_ij, lambda_m), ...]
    """
    n,l,j = s
    outs = []
    for lp in (l-1, l+1):
        if lp < 0:
            continue
        for jp in (lp-0.5, lp+0.5):
            if jp <= 0:
                continue
            # n' 범위: [n_floor .. n-1]
            for np_ in range(max(n_floor,5), n):
                try:
                    A = atom.getTransitionRate(n,l,j, np_,lp,jp)  # [s^-1]
                except Exception:
                    A = 0.0
                if A > 0.0:
                    lam_m = abs(atom.getTransitionWavelength(n,l,j, np_,lp,jp))  # [m]
                    outs.append(((np_,lp,jp), A, lam_m))
    return outs


셀 4 — 실행 & 결과 집계/시각화

In [ ]:
atom = make_atom(ATOM)
print(f"Start: {state_str(START_STATE)}  |  ATOM={ATOM}")

# 해석적(무작위 없이) 기대 스펙트럼
lines = branching_spectrum(atom, START_STATE,
                           max_steps=MAX_STEPS,
                           min_weight=MIN_WEIGHT,
                           n_floor=N_FLOOR)

# DataFrame으로 정리
df = pd.DataFrame(lines, columns=["lambda_nm", "weight"])
# 가시권 필터
df_vis = df.query(f"{VISIBLE_RANGE[0]} <= lambda_nm <= {VISIBLE_RANGE[1]}").copy()

# 500–600 nm 대역에서의 평균 광자수(= 변환 수율)
yield_500_600 = df_vis.query(f"{COUNT_WINDOW[0]} <= lambda_nm <= {COUNT_WINDOW[1]}")["weight"].sum()

print(f"라인 개수(전체): {len(df)}")
print(f"가시권({VISIBLE_RANGE[0]}–{VISIBLE_RANGE[1]} nm) 라인 개수: {len(df_vis)}")
print(f"500–600 nm 평균 광자수/여기 = {yield_500_600:.4f}")

# 히스토그램(가시권)
bins = np.arange(VISIBLE_RANGE[0], VISIBLE_RANGE[1] + BIN_WIDTH_NM, BIN_WIDTH_NM)
hist, edges = np.histogram(df_vis["lambda_nm"], bins=bins, weights=df_vis["weight"])
centers = 0.5 * (edges[:-1] + edges[1:])

fig, ax = plt.subplots(figsize=(8,4))
ax.bar(centers, hist, width=BIN_WIDTH_NM, align="center")
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("Expected photons per excitation (arb.)")
ax.set_title(f"Branching-ratio spectrum from {state_str(START_STATE)}")
plt.show()

# 피크 상위 20개 라인 표
df_lines = (df_vis.groupby("lambda_nm", as_index=False)["weight"].sum()
                  .sort_values("weight", ascending=False)
                  .head(20)
                  .reset_index(drop=True))
df_lines


해석 & 활용 팁

weight는 초기 여기 1회당 그 파장 라인에서 기대되는 광자수입니다. 모든 라인을 합치면 평균 방출 광자수가 됩니다(모델 범위 내에서 E1만).

500–600 nm 평균 광자수/여기가 카메라 민감 대역에서의 THz→optical 변환 수율 지표로 유용합니다. 전이 후보(예: 상·하 상태)마다 위 코드를 한 번씩 돌려 수율이 큰 쪽을 고르세요.

속도/정밀 조절:

더 빠르게: MAX_STEPS를 8–10, MIN_WEIGHT를 1e-5 수준으로.

더 정밀하게: MAX_STEPS↑, MIN_WEIGHT↓, N_FLOOR↓(예: 4), 그리고 BIN_WIDTH_NM↓(0.5 nm).

주의: 이 모델은 자발 E1 붕괴만 고려합니다(ARC의 getTransitionRate가 그 값). BBR 유도 전이나 충돌 유도 전이까지 넣고 싶다면, 각 전이율에 해당 항을 추가해 
𝐴~=𝐴_sp+𝐴_BBR+𝐴_coll 
로 분기비를 다시 만들면 됩니다(원하시면 해당 버전도 만들어 드릴게요).